In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import functions as f

import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder.appName('log_reg').getOrCreate()

22/02/16 11:33:07 WARN Utils: Your hostname, ganesh-pi resolves to a loopback address: 127.0.1.1; using 192.168.1.119 instead (on interface eth0)
22/02/16 11:33:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/16 11:33:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')
df.show()

22/02/16 11:34:26 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
log_reg_model = LogisticRegression()

fitted_log_reg_model = log_reg_model.fit(df)

results = fitted_log_reg_model.summary

In [9]:
results.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [10]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514870...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198872...|[6.76550379997544...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234669...|[4.87842678719363...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012715...|[4.62137287300564...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874693...|[1.81823629114105...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504180...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212819...|[6.97903542820426...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446273...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606545...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [11]:
#splitting the data into training and testing

lr_train, lr_test = df.randomSplit([0.7, 0.3])

fitted_model = log_reg_model.fit(lr_train)

predictions_lr = fitted_model.evaluate(lr_test)
predictions_lr.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[22.2500872614395...|[0.99999999978277...|       0.0|
|  0.0|(692,[124,125,126...|[55.9296929979790...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[31.9096824889648...|[0.99999999999998...|       0.0|
|  0.0|(692,[126,127,128...|[38.2358314361694...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[15.4008887242905...|[0.99999979512973...|       0.0|
|  0.0|(692,[126,127,128...|[29.5762487462652...|[0.99999999999985...|       0.0|
|  0.0|(692,[127,128,129...|[19.8135866233609...|[0.99999999751647...|       0.0|
|  0.0|(692,[127,128,129...|[22.7204352587774...|[0.99999999986428...|       0.0|
|  0.0|(692,[152,153,154...|[37.5746768021352...|           [1.0,0.0]|       0.0|
|  0.0|(692,[153

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
evaluator.evaluate(predictions_lr.predictions)

0.9827694235588972